In [34]:
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from transformers import BertTokenizer, TFBertModel, BertConfig
import tensorflow as tf
import numpy as np
import pickle

## Import Data

In [2]:
ekg_denoised = pd.read_pickle('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised_v2.pkl')
ekg_denoised = ekg_denoised.sample(frac=1)

In [3]:
ekg_denoised.head()

,ICD9_CODE,TEXT
11614,3,sinus rhythm right bundle branch block since p...
21324,0,sinus rhythm. low limb lead voltage. complete ...
37593,1,atrial fibrillation. low limb lead voltage is ...
35194,1,wide complex tachycardia with right bundle-bra...
38909,1,sinus tachycardia. indeterminate axis. right v...


## Tokenize Data

In [4]:
tokenizer = hub.KerasLayer(
    'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

INFO:absl:Using /tmp/tfhub_modules to cache modules.


In [5]:
encoder = hub.KerasLayer(
    'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1')

In [6]:
batched = np.array_split(ekg_denoised, 5)

In [35]:
with open('/home/sanjaycollege15/PredictingDiagnoses/Data/smallBERT_dataset.pkl', 'wb') as f:
    pickle.dump(batched, f)

In [8]:
tokens = tokenizer(batched[0].TEXT)
encoded_inputs = encoder(tokens)

In [10]:
encoded_inputs['encoder_outputs']

list

In [30]:
encoded_inputs.keys()

dict_keys(['pooled_output', 'encoder_outputs', 'default', 'sequence_output'])

In [11]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM

In [12]:
inputs = encoded_inputs['encoder_outputs']

In [36]:
with open('/home/sanjaycollege15/PredictingDiagnoses/Data/smallBERT_encoder_outputs1.pkl', 'wb') as f:
    pickle.dump(inputs, f)

OSError: [Errno 28] No space left on device

In [ ]:
pickle.dump()

In [24]:
embeddings = inputs[0]

In [21]:
labels=np.array(batched[0].ICD9_CODE)

In [18]:
encoded_inputs.keys()

dict_keys(['default', 'sequence_output', 'pooled_output', 'encoder_outputs'])

In [31]:
xmodel= Sequential()
xmodel.add(SimpleRNN(128, input_shape=(128,512)))
xmodel.add(Dense(4, activation='relu'))
xmodel.build()
xmodel.summary()

xmodel.compile(optimizer='adam', 
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
               metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 128)               82048     
                                                                 
 dense_2 (Dense)             (None, 4)                 516       
                                                                 
Total params: 82,564
Trainable params: 82,564
Non-trainable params: 0
_________________________________________________________________


In [32]:
xmodel.fit(embeddings, labels, epochs=2, batch_size=512)

Epoch 1/2
27/27 [==============================] - 8s 232ms/step - loss: 1.4176 - accuracy: 0.2526
Epoch 2/2
27/27 [==============================] - 6s 213ms/step - loss: 1.3863 - accuracy: 0.2501


In [7]:
labels = np.array(ekg_denoised.ICD9_CODE[:1000])

In [13]:
encoded_inputs.keys()

dict_keys(['pooled_output', 'encoder_outputs', 'default', 'sequence_output'])

In [14]:
test_encodings = encoded_inputs['pooled_output']

<tf.Tensor: shape=(1000, 512), dtype=float32, numpy=
array([[ 0.99166214,  0.1197574 , -0.06184475, ...,  0.5502014 ,
        -0.6442353 , -0.85817146],
       [ 0.9976368 ,  0.17481573, -0.19261858, ...,  0.5003145 ,
        -0.5349502 , -0.1832304 ],
       [ 0.64804137, -0.44555837,  0.02937316, ...,  0.56327635,
        -0.11237483, -0.92120934],
       ...,
       [ 0.9978899 , -0.42364314, -0.20901826, ...,  0.47847232,
        -0.6196047 , -0.04929641],
       [ 0.9972557 , -0.8440902 , -0.31961453, ...,  0.28505862,
        -0.2737933 , -0.04500105],
       [ 0.9970064 , -0.759793  ,  0.05473077, ...,  0.4079414 ,
        -0.3602471 , -0.35015243]], dtype=float32)>

### Test Set

In [23]:
test_tokens = tokenizer(ekg_denoised.TEXT[1000:1100])
test_encodings = encoder(test_tokens)

In [19]:
test_labels = np.array(ekg_denoised.ICD9_CODE[1000:1100])

In [15]:
inputs = tf.keras.layers.Input(shape=(encoded_inputs['pooled_output'].shape[1],))
hidden_1_out = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(inputs)
dropout = tf.keras.layers.Dropout(0.2)(hidden_1_out)
hidden_2_out = tf.keras.layers.Dense(200, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(dropout)
classification_out = tf.keras.layers.Dense(4, activation='softmax')(hidden_2_out)

model_functional = tf.keras.models.Model(inputs=inputs, outputs = classification_out)
model_functional.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model_functional.fit(encoded_inputs['pooled_output'], labels, epochs=15, batch_size=512)


Epoch 1/15
2/2 [==============================] - 1s 11ms/step - loss: 1.8573 - accuracy: 0.2590
Epoch 2/15
2/2 [==============================] - 0s 10ms/step - loss: 1.7130 - accuracy: 0.2670
Epoch 3/15
2/2 [==============================] - 0s 9ms/step - loss: 1.6672 - accuracy: 0.2410
Epoch 4/15
2/2 [==============================] - 0s 9ms/step - loss: 1.6256 - accuracy: 0.2610
Epoch 5/15
2/2 [==============================] - 0s 9ms/step - loss: 1.5825 - accuracy: 0.3100
Epoch 6/15
2/2 [==============================] - 0s 10ms/step - loss: 1.6065 - accuracy: 0.3210
Epoch 7/15
2/2 [==============================] - 0s 9ms/step - loss: 1.5823 - accuracy: 0.3250
Epoch 8/15
2/2 [==============================] - 0s 9ms/step - loss: 1.5519 - accuracy: 0.3450
Epoch 9/15
2/2 [==============================] - 0s 9ms/step - loss: 1.5554 - accuracy: 0.3350
Epoch 10/15
2/2 [==============================] - 0s 10ms/step - loss: 1.5476 - accuracy: 0.3420
Epoch 11/15
2/2 [==================

In [26]:
model_functional.evaluate(test_encodings['pooled_output'], test_labels)


4/4 [==============================] - 0s 3ms/step - loss: 1.5750 - accuracy: 0.2600


[1.5749602317810059, 0.25999999046325684]

## Saving Pooled Outputs

In [ ]:
tokens = tokenizer(ekg_denoised.TEXT)
encoded_inputs = encoder(tokens)